# T2 - Demographics 

We've already seen a few basic ways to add vital dynamics to your model, e.g. 

In [ ]:
import starsim as ss

pars = dict(
    birth_rate=20,     # Annual crude birth rate (per 1000 people)
    death_rate=0.015,  # Annual crude death rate
    networks='random',
    diseases='sir'
)
sim = ss.Sim(pars)


This will scale all your results to represent a population of 50 million people, and will apply annual birth and death rates as specified in the `pars` dict. Alternatively, we can make demographic components:

In [ ]:
demographics = [
    ss.births(pars={'birth_rate': 20}),
    ss.background_deaths(pars={'death_rate': 0.015})
]
sim = ss.Sim(diseases=ss.SIR(), demographics=demographics, networks=ss.random())

Both achieve the same thing. 

## Scaling results to whole populations
Even though we've been simulating populations of a few thousand agents, we can also use the `total_pop` parameter to scale our results so that they reflect a much larger population. You can think of this as a kind of statistical sampling approximation. If we want to model the population of Nigeria, for example, it would be much too computationally intensive to simulate 200 million agents. However, we could simulate 50,000 agents and then say that each agent represents 4,000 people. Again, we can do this by passing `total_pop=200e6` to the sim or in the `pars` dictionary.

## Using realistic vital demographics
For more realistic demographics, we can also pass in a file that has birth or death rates over time and by age/sex. There are examples of these files in the `tests/test_data` folder. Here's how we would read them in and construct realistic vital dynamics for Nigeria: 

In [ ]:
import starsim as ss
import pandas as pd

# Read in age-specific fertility rates
fertility_rates = pd.read_csv(ss.root / 'tests/test_data/nigeria_asfr.csv')
pregnancy = ss.Pregnancy(pars={'fertility_rate': fertility_rates})

death_rates = pd.read_csv(ss.root / 'tests/test_data/nigeria_deaths.csv')
death = ss.background_deaths(pars={'death_rate': death_rates})

demographics = [pregnancy, death]

# Make people using the distribution of the population by age/sex in 1995
n_agents = 5_000
nga_pop_1995 = 106819805  # Population of Nigeria in 1995, the year we will start the model
age_data = pd.read_csv(ss.root / 'tests/test_data/nigeria_age.csv')
ppl = ss.People(n_agents, age_data=age_data)

# Define the parameters
pars = dict(
    people=ppl,  # People with the right age/sex structure
    demographics=demographics,  # Demographics 
    pars=dict(total_pop=nga_pop_1995, networks='random', diseases='sir'),  # Networks and diseases
)

# Make the sim, run and plot
sim = ss.Sim(pars)
sim.run()

# Read in a file with the actual population size
nigeria_popsize = pd.read_csv(ss.root / 'tests/test_data/nigeria_popsize.csv')
data = nigeria_popsize[(nigeria_popsize.year >= 1995) & (nigeria_popsize.year <= 2030)]

# Plot the overall population size - simulated vs data
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1)
ax.scatter(data.year, data.n_alive, alpha=0.5, label='Data')
ax.plot(sim.yearvec, sim.results.n_alive, color='k', label='Model')
ax.legend()
ax.set_title('Population')
plt.show();

If you want to use realistic demographics for your model, you can adapt the data files and code snippet above to read in the relevant demographic data files for your country/setting. 